In [1]:
from periodictable import elements
import numpy as np
import pickle

In [2]:
from exfor_tools import reaction, curate, quantities
import elm
from jitr import rmatrix

Using database version X4-2024-12-31 located in: /home/beyerk/db/exfor/unpack_exfor-2024/X4-2024-12-31


In [3]:
with open("nn_elastic_data.pkl", 'rb') as f:
    nn_data = pickle.load(f)

In [4]:
with open("pp_elastic_data.pkl", 'rb') as f:p
    pp_data = pickle.load(f)

In [9]:
all_data = list(pp_data.values()) + list(nn_data.values())
all_data_by_entry, sys_uncertainties_by_entry = (
    curate.cross_reference_entry_systematic_err(all_data)
)

In [13]:
for entry_id, entries in all_data_by_entry.items():
    for entry in entries:
        for measurement in entry.measurements:
            if measurement.systematic_offset_err > 0:
                # TODO
                measurement.systematic_offset_err = 0


In [14]:
from IPython.display import Latex, Math, display # just for fun
targets = sorted(list(set( list(nn_data.keys()) + list(pp_data.keys()))), key=lambda x:x[1] + x[0])
display(Math(", ".join([f"\,^{{{target[0]}}} \\rm{{{elements[target[1]]}}}" for target in targets])))

<IPython.core.display.Math object>

In [15]:
entry = pp_data[(40,20)].data['dXS/dA'].entries

In [16]:
measurement_groups = curate.categorize_measurements_by_energy( pp_data[(40,20)].data['dXS/dA'].entries)

In [17]:
constraints = []
for measurement_group in measurement_groups:
    constraints.append(
        elm.DifferentialElasticConstraint(
            reaction=pp_data[(40,20)].reaction,
            quantity='dXS/dA',
            measurement=measurement_group[0],
            angles_vis=np.linspace(0.01,180,100),
            core_solver=rmatrix.Solver(40),
            lmax=20,
            divide_by_Rutherford=False,       
        )
    )